<a href="https://colab.research.google.com/github/ACRainieri/pyspark/blob/main/Projeto_WEB_DSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Flask


In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route("/")
def index():
  return "Index!"

@app.route("/hello")
def hello():
  return "Hello World!!"

@app.route("/members")
def members():
  return "Members"

@app.route("/members/<string:name>/")  
def getMember(name):
  return name

if __name__ == "__main__":
  app.run(host='0.0.0.0', port=80)